In [ ]:
# @markdown # 1. Environment Configuration 🔧
# @markdown This section will configure all necessary environments for molecular docking in Google Colab.
# @markdown ### Instructions:
# @markdown 1. Click **Run** button on the left ▶️
# @markdown 2. Wait for "Environment configuration completed" message ⏱️✅
# @markdown 3. Restart runtime if prompted 🔄

import os
import sys
import subprocess
from pathlib import Path
import time

def clean_old_installations():
    print("Cleaning old installations...")
    commands = [
        "apt-get remove -y python2.7 python-pip python2.7-pip || true",
        "apt-get autoremove -y || true",
        "rm -rf /usr/local/bin/pip* || true",
        "rm -rf ~/.cache/pip || true",
        "apt-get clean",
        "apt-get update"
    ]

    all_success = True
    for cmd in commands:
        success, out, err = run_command(cmd)
        if not success:
            print(f"Warning during cleanup: {err}")
            all_success = False

    if not all_success:
        print("Some cleanup commands failed but continuing anyway...")

    print("Cleanup completed.")
    return True

def run_command(command, check=True, shell=True):
    try:
        result = subprocess.run(
            command,
            shell=shell,
            check=False, 
            capture_output=True,
            text=True
        )
        if result.returncode != 0 and check:
            print(f"Command failed: {command}")
            print(f"Output: {result.stdout}")
            print(f"Error: {result.stderr}")
            return False, result.stdout, result.stderr
        return True, result.stdout, result.stderr
    except Exception as e:
        print(f"Exception running command: {command}")
        print(f"Error: {str(e)}")
        return False, "", str(e)

def install_base_dependencies():
    print("\nInstalling base dependencies...")
    commands = [
        "apt-get update",
        "apt-get install -y software-properties-common",
        "add-apt-repository ppa:deadsnakes/ppa -y",
        "apt-get update",

        "apt-get install -y python2.7 python2.7-dev",

        "apt-get install -y csh wget tar",

        "ln -sf /usr/bin/python2.7 /usr/bin/python2"
    ]

    for cmd in commands:
        success, out, err = run_command(cmd)
        if not success:
            print(f"Error during base installation: {err}")
            return False

    success, out, err = run_command("python2 --version")
    if not success or "Python 2.7" not in (out + err):
        print("Python 2.7 installation verification failed")
        return False

    print("Base dependencies installed successfully.")
    return True

def setup_pip2():
    """设置 pip2"""
    print("\nSetting up pip2...")
    commands = [
        "curl -O https://bootstrap.pypa.io/pip/2.7/get-pip.py",
        "python2.7 get-pip.py --force-reinstall",

        "ln -sf /usr/local/bin/pip2.7 /usr/bin/pip2",

        "python2.7 -m pip install --upgrade 'pip>=20.3.4,<21.0'",

        "python2.7 -m pip install 'setuptools<45.0.0'"
    ]

    for cmd in commands:
        success, out, err = run_command(cmd)
        if not success:
            print(f"Error during pip2 setup: {err}")
            return False

    success, out, err = run_command("pip2 --version")
    if not success:
        print("pip2 installation verification failed")
        return False

    print("pip2 setup completed successfully.")
    return True

def setup_conda():
    """Set Conda Environment"""
    print("\nSetting up Conda environment...")

    if not Path('/usr/local/bin/conda').exists():
        run_command("wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh")
        run_command("chmod +x Miniconda3-latest-Linux-x86_64.sh")
        run_command("./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local")

        sys.path.append('/usr/local/bin')
        os.environ['PATH'] = '/usr/local/bin:' + os.environ['PATH']

    commands = [
        "conda create -n docking_env python=3.7 -y",
        "conda install -n docking_env -c conda-forge openbabel rdkit numpy scipy matplotlib pandas seaborn scikit-learn biopython -y"
    ]

    for cmd in commands:
        success, out, err = run_command(cmd)
        if not success:
            print(f"Error during conda setup: {err}")
            return False

    print("Conda environment setup completed.")
    return True

def install_python2_packages():
    print("\nInstalling Python 2.7 packages...")

    try:
        run_command("python2.7 -m pip install numpy==1.16.6")

        success, out, err = run_command(
            "python2.7 -c 'import numpy; print(numpy.__version__)'")
        if not success:
            raise Exception("numpy installation failed")

        test_script = """
import numpy
import sys
print('Numpy version:', numpy.__version__)
print('Python version:', sys.version)
        """

        with open('test_py27.py', 'w') as f:
            f.write(test_script)

        success, out, err = run_command("python2.7 test_py27.py")
        if not success:
            raise Exception("Failed to run test script")

        print("Python 2.7 packages installed successfully.")
        return True

    except Exception as e:
        print(f"Error installing Python 2.7 packages: {e}")
        return False

def install_mgltools():

    print("\nInstalling MGLTools...")

    try:
        if not Path('/usr/local/autodocktools/bin/pythonsh').exists():
            run_command("wget https://ccsb.scripps.edu/mgltools/download/491/tars/releases/REL1.5.7/mgltools_x86_64Linux2_1.5.7.tar.gz -O autodocktools.tar.gz")
            run_command("mkdir -p /usr/local/autodocktools")
            run_command("tar -xzf autodocktools.tar.gz -C /usr/local/autodocktools --strip-components=1")
            run_command("tar -xzf /usr/local/autodocktools/MGLToolsPckgs.tar.gz -C /usr/local/autodocktools/")

            pythonsh_content = """#!/bin/bash
/usr/bin/python2.7 "$@"
"""
            with open("/usr/local/autodocktools/bin/pythonsh", "w") as f:
                f.write(pythonsh_content)
            run_command("chmod +x /usr/local/autodocktools/bin/pythonsh")

        print("MGLTools installation completed.")
        return True

    except Exception as e:
        print(f"Error installing MGLTools: {e}")
        return False

def setup_environment_variables():
    print("\nSetting up environment variables...")

    try:
        os.environ['PYTHONPATH'] = "/usr/local/autodocktools/MGLToolsPckgs"
        os.environ['PATH'] = '/usr/bin:/usr/local/bin:/usr/local/autodocktools/bin:' + os.environ['PATH']

        success, out, err = run_command("python2 --version")
        if not success:
            raise Exception("Python 2.7 is not properly configured")

        global mgltools_path, prepare_receptor4_path, prepare_ligand4_path
        mgltools_path = "/usr/local/autodocktools/bin/pythonsh"
        prepare_receptor4_path = "/usr/local/autodocktools/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_receptor4.py"
        prepare_ligand4_path = "/usr/local/autodocktools/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_ligand4.py"

        print("Environment variables set successfully.")
        return True

    except Exception as e:
        print(f"Error setting environment variables: {e}")
        return False

def verify_installation():
    print("\nVerifying installation...")

    checks = [
        ("Python 2.7", "python2.7 --version"),
        ("pip2", "pip2 --version"),
        ("numpy (Python 2.7)", "python2.7 -c 'import numpy; print(numpy.__version__)'"),
        ("MGLTools pythonsh", "test -x /usr/local/autodocktools/bin/pythonsh && echo 'exists and executable'"),
    ]

    all_passed = True
    for name, cmd in checks:
        success, out, err = run_command(cmd)
        if success:
            print(f"✓ {name} verified: {out.strip() or err.strip()}")
        else:
            print(f"✗ {name} verification failed")
            all_passed = False

    paths = {
        'MGLTools': mgltools_path,
        'Prepare Receptor': prepare_receptor4_path,
        'Prepare Ligand': prepare_ligand4_path
    }

    for name, path in paths.items():
        if Path(path).exists():
            print(f"✓ {name} path verified: {path}")
        else:
            print(f"✗ {name} path not found: {path}")
            all_passed = False

    return all_passed

def persist_environment():
    print("\nSaving environment settings...")

    env_settings = """
import os
import sys
import subprocess

# MGLTools paths
mgltools_path = "/usr/local/autodocktools/bin/pythonsh"
prepare_receptor4_path = "/usr/local/autodocktools/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_receptor4.py"
prepare_ligand4_path = "/usr/local/autodocktools/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_ligand4.py"

# Environment variables
os.environ['PYTHONPATH'] = "/usr/local/autodocktools/MGLToolsPckgs"
os.environ['PATH'] = '/usr/local/autodocktools/bin:' + os.environ['PATH']
"""

    try:
        with open('/content/env_settings.py', 'w') as f:
            f.write(env_settings)
        print("Environment settings saved successfully.")
        return True
    except Exception as e:
        print(f"Error saving environment settings: {e}")
        return False

def main():
    print("Starting environment configuration...\n")
    start_time = time.time()

    steps = [
        ("Cleaning old installations", clean_old_installations),
        ("Installing base dependencies", install_base_dependencies),
        ("Setting up pip2", setup_pip2),
        ("Setting up Conda", setup_conda),
        ("Installing Python 2.7 packages", install_python2_packages),
        ("Installing MGLTools", install_mgltools),
        ("Setting up environment variables", setup_environment_variables),
        ("Persisting environment", persist_environment)
    ]

    max_retries = 3
    for step_name, step_func in steps:
        for attempt in range(max_retries):
            try:
                print(f"\n{'='*20} {step_name} {'='*20}")
                if not step_func():
                    raise Exception(f"{step_name} failed")
                break
            except Exception as e:
                if attempt < max_retries - 1:
                    print(f"\n⚠️ Attempt {attempt + 1} failed: {str(e)}")
                    print("Retrying...")
                    time.sleep(2)  # 短暂延迟后重试
                else:
                    print(f"\n❌ {step_name} failed after {max_retries} attempts: {str(e)}")
                    return False

    if verify_installation():
        elapsed_time = time.time() - start_time
        print(f"\n✅ Environment configuration completed successfully in {elapsed_time:.1f} seconds!")
        print("\nNote: You may need to restart the runtime for all changes to take effect.")
        return True
    else:
        print("\n❌ Installation verification failed.")
        print("Please check the error messages above and try again.")
        return False

if __name__ == "__main__":
    main()